<a href="https://colab.research.google.com/github/akariiijima/AdversarialExample/blob/master/kerasCNN_AE1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import keras
import numpy as np
import pandas as pd
import tensorflow as tf

In [0]:
df=pd.read_csv("mnist_train.csv")
#1-10 labels
labels=df["5"].values
#one-hot labels
labels = tf.keras.utils.to_categorical(labels)

df.drop("5", axis=1, inplace=True)
pixels=df.values
pixels=pixels/255
pixels=pixels.reshape(len(df),28,28,1)

In [0]:
class Adversarial_Training(keras.models.Model):
    _at_loss = None
    # set up
    def setup_at_loss(self, loss_func=keras.losses.categorical_crossentropy, eps=0.25/255.0):
        self._loss_func = loss_func
        self._at_loss = self.at_loss(eps)
        return self
    # VAT loss
    def at_loss(self, eps):
      
        # original loss
        loss_orig = self._loss_func(self.inputs[1], self.outputs[0])
        
        # gradients
        map_grads = keras.backend.stop_gradient(keras.backend.gradients(loss_orig, self.inputs[0]))
        list_grads = list(map_grads)
        grads = list_grads[0]
        
        #grads = keras.backend.gradients(loss_orig, self.inputs[0])[0]
        #loss = F.softmax_cross_entropy(chainer_model(target_array), Variable(np.array([orig_ind.astype(np.int32)])))
        #adv_array = self.inputs[0] + eps * keras.backend.sign(target_array.grad)
        
        # perterbed samples
        new_inputs = self.inputs[0] - eps * keras.backend.sign(grads[0])
        
        # estimation for the perturbated samples
        outputs_perturb = self.call([new_inputs, self.inputs[1]])
        
        # computing losses
        loss = self._loss_func(self.inputs[1], outputs_perturb)
       
        return loss


In [0]:
input_pixels=keras.layers.Input(shape=(28, 28,1))
model_cnn=keras.layers.Conv2D(20, (3, 3), padding="valid", activation=tf.nn.relu,)(input_pixels)
model_cnn=keras.layers.MaxPooling2D(pool_size=(3,3))(model_cnn)
model_cnn=keras.layers.Conv2D(50, (3, 3), padding="valid", activation=tf.nn.relu,)(model_cnn)
model_cnn=keras.layers.MaxPooling2D(pool_size=(3,3))(model_cnn)
model_cnn=keras.layers.BatchNormalization()(model_cnn)
model_cnn=keras.layers.Flatten()(model_cnn)
model_cnn=keras.layers.Dense(225, activation=tf.nn.relu)(model_cnn)
model_cnn=keras.layers.Dense(10, activation="softmax")(model_cnn)

In [323]:
# adversarial training
input_label = keras.layers.Input(shape=(10,))
model = Adversarial_Training(input=[input_pixels, input_label], output=model_cnn).setup_at_loss(loss_func=keras.losses.categorical_crossentropy, eps=0.25)
print("input_pixels" , input_pixels.shape)
print("input_label" , input_label.shape)
print("model_cnn" , model_cnn.shape)
print(model.output)
#cnn=keras.models.Model(input=input_pixels, output=cnn)
#cnn.compile(optimizer=tf.train.AdamOptimizer(), loss="categorical_crossentropy", metrics=["accuracy"])
#print(cnn.predict(pixels).shape)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Update your `Adversarial_Training` call to the Keras 2 API: `Adversarial_Training(inputs=[<tf.Tenso..., outputs=Tensor("de...)`
  


input_pixels (?, 28, 28, 1)
input_label (?, 10)
model_cnn (?, 10)
Tensor("dense_92/Softmax:0", shape=(?, 10), dtype=float32)


In [324]:
model.compile(optimizer=tf.train.AdamOptimizer(), loss="categorical_crossentropy", metrics=["accuracy"])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_130 (InputLayer)       (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_91 (Conv2D)           (None, 26, 26, 20)        200       
_________________________________________________________________
max_pooling2d_91 (MaxPooling (None, 8, 8, 20)          0         
_________________________________________________________________
conv2d_92 (Conv2D)           (None, 6, 6, 50)          9050      
_________________________________________________________________
max_pooling2d_92 (MaxPooling (None, 2, 2, 50)          0         
_________________________________________________________________
batch_normalization_46 (Batc (None, 2, 2, 50)          200       
_________________________________________________________________
flatten_46 (Flatten)         (None, 200)               0         
__________

In [326]:
print(pixels.shape)
print(labels.shape)
print(labels)
#model.predict([pixels, labels])
#print(model.predict([pixels, labels]).shape)
fit=model.fit([pixels, labels],labels,epochs=50, batch_size=4096,validation_split=0.2)

(59999, 28, 28, 1)
(59999, 10)
[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]]
Train on 47999 samples, validate on 12000 samples
Epoch 1/50
47999/47999 [==============================] - 8s 165us/step - loss: 1.3945 - acc: 0.6110 - val_loss: 0.5939 - val_acc: 0.8581
Epoch 2/50
47999/47999 [==============================] - 2s 44us/step - loss: 0.4197 - acc: 0.8920 - val_loss: 0.2818 - val_acc: 0.9110
Epoch 3/50
47999/47999 [==============================] - 2s 44us/step - loss: 0.2221 - acc: 0.9332 - val_loss: 0.2007 - val_acc: 0.9373
Epoch 4/50
47999/47999 [==============================] - 2s 44us/step - loss: 0.1518 - acc: 0.9542 - val_loss: 0.1761 - val_acc: 0.9439
Epoch 5/50
47999/47999 [==============================] - 2s 44us/step - loss: 0.1158 - acc: 0.9656 - val_loss: 0.1652 - val_acc: 0.9478
Epoch 6/50
47999/47999 [==============================] - 2s 44us/step - loss

In [0]:
df_test=pd.read_csv("mnist_test.csv")
#1-10 labels
labels_test=df_test["7"].values
#one-hot labels
labels_test = tf.keras.utils.to_categorical(labels_test)

df_test.drop("7", axis=1, inplace=True)
pixels_test=df_test.values
pixels_test=pixels_test/255
pixels_test=pixels_test.reshape(len(df_test),28,28,1)

In [328]:
adversarialExample_test = np.tile(np.array([1.,0.,0.,0.,0.,0.,0.,0.,0.,0.]),(len(df_test),1))
test_loss, test_acc = model.evaluate([pixels_test, labels_test],labels_test)

print("Test_loss:", test_loss)
print("Test_accuracy:", test_acc)

9999/9999 [==============================] - 1s 80us/step
Test_loss: 0.0326302270295655
Test_accuracy: 0.9906990699069907
